In [2]:
"""
Coffee Shop Employee Scheduling Optimization
MILP Project - Linear Programming & Optimization Class

This notebook contains:
1. Complete mathematical formulation
2. Simulated realistic data
3. Gurobi implementation
4. Results analysis and visualization
"""

print("COFFEE SHOP EMPLOYEE SCHEDULING OPTIMIZATION")

import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(3230)

print("MATHEMATICAL FORMULATION")

formulation = """
SETS:
  E = Set of employees (e = 1, 2, ..., 15)
  S = Set of shifts (s = Monday 7-11AM, Monday 11AM-3PM, ..., Sunday 7-12AM)
  K = Set of skills (k = Barista, Cashier, Shift Lead, Food Prep, Cleaner)

PARAMETERS:
  w_e = Hourly wage of employee e ($/hour)
  h_s = Duration of shift s (hours)
  p_es = Preference score of employee e for shift s [1-5]
  a_es = Availability: 1 if employee e can work shift s, 0 otherwise
  skill_ek = 1 if employee e has skill k, 0 otherwise
  req_sk = Number of employees with skill k required for shift s
  maxH_e = Maximum weekly hours for employee e (hours)
  λ = Weight parameter for cost vs. preference trade-off [0,1]

DECISION VARIABLES:
  x_es ∈ {0,1} = 1 if employee e is assigned to shift s, 0 otherwise

OBJECTIVE FUNCTION:
  Minimize: λ × (Total Cost) - (1 - λ) × (Total Preference)
  
  Where:
    Total Cost = Σ_e Σ_s (x_es × w_e × h_s)
    Total Preference = Σ_e Σ_s (x_es × p_es)

CONSTRAINTS:
  1. Skill Coverage: Σ_e (x_es × skill_ek) ≥ req_sk  ∀s ∈ S, ∀k ∈ K
     (Each shift must have enough employees with required skills)
  
  2. Availability: x_es ≤ a_es  ∀e ∈ E, ∀s ∈ S
     (Employees can only work when available)
  
  3. Maximum Hours: Σ_s (x_es × h_s) ≤ maxH_e  ∀e ∈ E
     (Total hours cannot exceed maximum weekly hours)
  
  4. No Overlap: Σ_{s∈Overlap(t)} x_es ≤ 1  ∀e ∈ E, ∀t ∈ Time periods
     (Employee cannot work overlapping shifts)
"""

print(formulation)

print("PART 2: DATA GENERATION")

print("\n 2.1: Defining Skills")
skills = ['Barista', 'Cashier', 'Shift Lead', 'Food Prep', 'Cleaner']
print(f"Skills available: {skills}")

print("\n 2.2: Generating Shifts")

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
time_slots = [
    ('7-11AM', 7, 11, 4),
    ('11AM-3PM', 11, 15, 4),
    ('3-7PM', 15, 19, 4),
    ('7-12AM', 19, 24, 5)
]

# Generate all shifts
shifts = []
shift_hours = {}
shift_times = {}

for day in days:
    for slot_name, start, end, hours in time_slots:
        shift_name = f"{day} {slot_name}"
        shifts.append(shift_name)
        shift_hours[shift_name] = hours
        shift_times[shift_name] = (day, start, end)

print(f"Total shifts per week: {len(shifts)}")
print(f"Sample shifts: {shifts[:4]}")
print(f"\nShift durations:")
for shift in shifts[:4]:
    print(f"  {shift}: {shift_hours[shift]} hours")

print("\n 2.3: Generating Employee Data")

num_employees = 15
employees = {}

# Creating employees
employee_names = [
    "Alice", "Bob", "Charlie", "Diana", "Ethan",
    "Fiona", "George", "Hannah", "Isaac", "Julia",
    "Kevin", "Laura", "Mike", "Nina", "Oscar"
]

# skill pool to make sure there is proper coverage
skill_pool = {
    'Barista': ['Alice', 'Bob', 'Charlie', 'Diana', 'Ethan', 'Fiona', 'George', 'Hannah', 'Isaac', 'Julia'],
    'Cashier': ['Alice', 'Bob', 'Charlie', 'Diana', 'Ethan', 'Fiona', 'Kevin', 'Laura', 'Mike', 'Nina'],
    'Shift Lead': ['Bob', 'Diana', 'George', 'Julia', 'Laura', 'Oscar'],
    'Food Prep': ['Charlie', 'Ethan', 'Hannah', 'Isaac', 'Kevin', 'Mike', 'Nina'],
    'Cleaner': ['Fiona', 'George', 'Hannah', 'Kevin', 'Mike', 'Nina', 'Oscar']
}

for name in employee_names:
    employee_skills = [skill for skill, emps in skill_pool.items() if name in emps]
    
    employees[name] = {
        'wage': 15,  # $15/hour for all
        'skills': employee_skills,
        'max_hours': 20  # 20 hours per week max
    }

# employee data
print(f"Total employees: {num_employees}")
print("\nEmployee Details:")
df_employees = pd.DataFrame([
    {
        'Employee': name,
        'Wage': f"${data['wage']}/hr",
        'Skills': ', '.join(data['skills']),
        'Max Hours': data['max_hours']
    }
    for name, data in employees.items()
])
print(df_employees.to_string(index=False))

# Check distribution of skills
print("\nSkill Distribution:")
skill_counts = {skill: 0 for skill in skills}
for emp_data in employees.values():
    for skill in emp_data['skills']:
        skill_counts[skill] += 1
for skill, count in skill_counts.items():
    print(f"  {skill}: {count} employees")

print("\n 2.4: Generating Availability Matrix ")

availability = {}

# availability to make sure this is feasibile
for emp_name in employees:
    availability[emp_name] = {}
    for shift in shifts:
        # Start with high availability (80%)
        available = np.random.choice([0, 1], p=[0.20, 0.80])
        
        # Students slightly less available during weekday mornings
        if shift.split()[0] in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            if '7-11AM' in shift or '11AM-3PM' in shift:
                if np.random.random() < 0.25:  # 25% reduction
                    available = 0
        
        availability[emp_name][shift] = available

# make sure each employee is available for at least 15 shifts
for emp_name in employees:
    available_count = sum(availability[emp_name].values())
    if available_count < 15:
        unavailable_shifts = [s for s in shifts if availability[emp_name][s] == 0]
        needed = 15 - available_count
        to_make_available = np.random.choice(unavailable_shifts, min(needed, len(unavailable_shifts)), replace=False)
        for shift in to_make_available:
            availability[emp_name][shift] = 1

# statistics
avg_availability = np.mean([sum(availability[e].values()) for e in employees])
print(f"Average shifts available per employee: {avg_availability:.1f} out of {len(shifts)}")
print(f"Average availability rate: {avg_availability/len(shifts)*100:.1f}%")

# Create availability DataFrame
df_availability = pd.DataFrame(availability).T
print(f"\nAvailability Matrix Shape: {df_availability.shape}")
print("Sample (first 5 employees, first 4 shifts):")
print(df_availability.iloc[:5, :4])

print("\n 2.5: Generating Preference Scores")

preferences = {}
for emp_name in employees:
    preferences[emp_name] = {}
    for shift in shifts:
        if availability[emp_name][shift] == 1:
            # base preferences
            base_pref = np.random.choice([1, 2, 3, 4, 5], p=[0.1, 0.15, 0.25, 0.3, 0.2])
            
            # preference for weekend shifts
            if 'Saturday' in shift or 'Sunday' in shift:
                base_pref = min(5, base_pref + np.random.choice([0, 1], p=[0.5, 0.5]))
            
            # preference for evening shifts
            if '7-12AM' in shift:
                base_pref = min(5, base_pref + np.random.choice([0, 1], p=[0.6, 0.4]))
            
            preferences[emp_name][shift] = base_pref
        else:
            preferences[emp_name][shift] = 0  # No preference for unavailable shifts

# statistics
all_prefs = [preferences[e][s] for e in employees for s in shifts if availability[e][s] == 1]
print(f"Average preference score (for available shifts): {np.mean(all_prefs):.2f}")
print(f"Preference distribution:")
for score in [1, 2, 3, 4, 5]:
    count = sum(1 for p in all_prefs if p == score)
    print(f"  Score {score}: {count} ({count/len(all_prefs)*100:.1f}%)")

print("SIMULATED DATA")

# 2.6: Defining Shift Requirements
shift_requirements = {}

for shift in shifts:
    day = shift.split()[0]
    time = shift.split()[1]
    
    is_weekend = day in ['Saturday', 'Sunday']
    is_morning = time == '7-11AM'
    is_lunch = time == '11AM-3PM'
    is_evening = time == '3-7PM'
    is_night = time == '7-12AM'
    
    # Conservative requirements
    if is_weekend:
        if is_morning or is_lunch:
            # Weekend brunch rush
            shift_requirements[shift] = {
                'Barista': 2,
                'Cashier': 1,
                'Shift Lead': 1,
                'Food Prep': 1,
                'Cleaner': 0
            }
        elif is_evening or is_night:
            # Weekend evening
            shift_requirements[shift] = {
                'Barista': 2,
                'Cashier': 1,
                'Shift Lead': 1,
                'Food Prep': 0,
                'Cleaner': 1
            }
    else:  # Weekday
        if is_morning:
            # Morning rush
            shift_requirements[shift] = {
                'Barista': 2,
                'Cashier': 1,
                'Shift Lead': 1,
                'Food Prep': 0,
                'Cleaner': 0
            }
        elif is_lunch:
            # Lunch rush
            shift_requirements[shift] = {
                'Barista': 2,
                'Cashier': 1,
                'Shift Lead': 1,
                'Food Prep': 0,
                'Cleaner': 0
            }
        elif is_evening:
            # Evening crowd
            shift_requirements[shift] = {
                'Barista': 1,
                'Cashier': 1,
                'Shift Lead': 1,
                'Food Prep': 0,
                'Cleaner': 1
            }
        elif is_night:
            # Night shift
            shift_requirements[shift] = {
                'Barista': 1,
                'Cashier': 1,
                'Shift Lead': 1,
                'Food Prep': 0,
                'Cleaner': 0
            }

# see feasibility
infeasible = []

for shift in shifts:
    for skill in skills:
        req = shift_requirements[shift].get(skill, 0)
        if req > 0:
            available = sum(
                1 for emp in employees
                if skill in employees[emp]['skills'] and availability[emp][shift] == 1
            )
            if available < req:
                infeasible.append((shift, skill, req, available))

# adjust infeasible requirements
for shift, skill, req, available in infeasible:
    shift_requirements[shift][skill] = max(0, available)

# --- MOVE df_requirements CREATION HERE AFTER shift_requirements IS DEFINED ---
rows = []
for shift in shifts:
    for skill in skills:
        req = shift_requirements[shift].get(skill, 0)
        rows.append({
            'Shift': shift,
            'Day': shift.split()[0],
            'Time': shift.split()[1],
            'Skill': skill,
            'Required': req
        })
df_requirements = pd.DataFrame(rows)
print("\nShift requirements (sample):")
print(df_requirements.head())

print("PART 3: BUILDING GUROBI OPTIMIZATION MODEL")

print("\n 3.1: Creating Model")
model = gp.Model("CoffeeShop_Scheduling")
model.Params.TimeLimit = 300
model.Params.MIPGap = 0.01
print("Model created: CoffeeShop_Scheduling")

print("\n 3.2: Adding Decision Variables")

x = {}
for emp_name in employees:
    for shift in shifts:
        x[emp_name, shift] = model.addVar(vtype=GRB.BINARY, 
                                          name=f"x_{emp_name}_{shift.replace(' ', '_').replace('-', '_')}")

print(f"Binary variables created: {len(x)}")
print(f"  ({num_employees} employees × {len(shifts)} shifts = {num_employees * len(shifts)} variables)")

print("\n 3.3: Setting Objective Function")

lambda_param = 0.6  # 60% cost, 40% preference

# Total cost
total_cost = gp.quicksum(
    x[emp_name, shift] * employees[emp_name]['wage'] * shift_hours[shift]
    for emp_name in employees
    for shift in shifts
)

# Total preference
total_preference = gp.quicksum(
    x[emp_name, shift] * preferences[emp_name][shift]
    for emp_name in employees
    for shift in shifts
)

# Normalize preference
max_possible_pref = num_employees * 5 * 5
preference_normalized = (total_preference / max_possible_pref) * 1000

# Multi-objective
objective = lambda_param * total_cost - (1 - lambda_param) * preference_normalized

model.setObjective(objective, GRB.MINIMIZE)

print(f"Objective function set:")
print(f"  λ (cost weight) = {lambda_param}")
print(f"  (1-λ) (preference weight) = {1 - lambda_param}")

print("\n 3.4: Adding Skill Coverage Constraints")

skill_constraints = 0
for shift in shifts:
    for skill in skills:
        required_count = shift_requirements[shift].get(skill, 0)
        if required_count > 0:
            model.addConstr(
                gp.quicksum(
                    x[emp_name, shift]
                    for emp_name in employees
                    if skill in employees[emp_name]['skills']
                ) >= required_count,
                name=f"skill_{shift.replace(' ', '_').replace('-', '_')}_{skill}"
            )
            skill_constraints += 1

print(f"Skill coverage constraints added: {skill_constraints}")

print("\n 3.5: Adding Availability Constraints")

avail_constraints = 0
for emp_name in employees:
    for shift in shifts:
        if availability[emp_name][shift] == 0:
            model.addConstr(
                x[emp_name, shift] == 0,
                name=f"avail_{emp_name}_{shift.replace(' ', '_').replace('-', '_')}"
            )
            avail_constraints += 1

print(f"Availability constraints added: {avail_constraints}")

print("\n 3.6: Adding Maximum Hours Constraints")

for emp_name in employees:
    model.addConstr(
        gp.quicksum(
            x[emp_name, shift] * shift_hours[shift]
            for shift in shifts
        ) <= employees[emp_name]['max_hours'],
        name=f"maxhours_{emp_name}"
    )

print(f"Maximum hours constraints added: {num_employees}")
print(f"  (Each employee limited to {employees['Alice']['max_hours']} hours/week)")

print("\n 3.7: Adding No-Overlap Constraints")

overlap_constraints = 0
for emp_name in employees:
    for day in days:
        day_shifts = [s for s in shifts if s.startswith(day)]
        
        for i, shift1 in enumerate(day_shifts):
            for shift2 in day_shifts[i+1:]:
                day1, start1, end1 = shift_times[shift1]
                day2, start2, end2 = shift_times[shift2]
                
                if not (end1 <= start2 or end2 <= start1):
                    model.addConstr(
                        x[emp_name, shift1] + x[emp_name, shift2] <= 1,
                        name=f"overlap_{emp_name}_{shift1.replace(' ', '_')}_{shift2.replace(' ', '_')}"
                    )
                    overlap_constraints += 1

print(f"No-overlap constraints added: {overlap_constraints}")

# Model summary
print("MODEL SUMMARY:")
print(f"  Decision Variables: {model.NumVars}")
print(f"  Constraints: {model.NumConstrs}")
print(f"  Binary Variables: {model.NumBinVars}")

print("PART 4: SOLVING THE MODEL")
model.optimize()

print("PART 5: RESULTS ANALYSIS")

if model.status == GRB.OPTIMAL:
    # Extract solution
    schedule = []
    for emp_name in employees:
        for shift in shifts:
            if x[emp_name, shift].X > 0.5:
                schedule.append({
                    'Employee': emp_name,
                    'Shift': shift,
                    'Day': shift.split()[0],
                    'Time': shift.split()[1],
                    'Hours': shift_hours[shift],
                    'Wage': employees[emp_name]['wage'],
                    'Cost': employees[emp_name]['wage'] * shift_hours[shift],
                    'Preference': preferences[emp_name][shift],
                    'Skills': ', '.join(employees[emp_name]['skills'])
                })

    df_schedule = pd.DataFrame(schedule)
    
    # 5.1: Schedule Overview
    print("\n 5.1: Schedule Overview ")
    print(f"Total assignments: {len(df_schedule)}")
    print(f"Total labor cost: ${df_schedule['Cost'].sum():.2f}")
    print(f"Total hours scheduled: {df_schedule['Hours'].sum():.0f} hours")
    print(f"Average preference score: {df_schedule['Preference'].mean():.2f}/5.0")
    
    # 5.2: Employee Workload
    print("\n 5.2: Employee Workload Distribution")
    employee_stats = df_schedule.groupby('Employee').agg({
        'Hours': 'sum',
        'Cost': 'sum',
        'Shift': 'count',
        'Preference': 'mean'
    }).round(2)
    employee_stats.columns = ['Total Hours', 'Total Cost', 'Shifts Assigned', 'Avg Preference']
    employee_stats = employee_stats.sort_values('Total Hours', ascending=False)
    print(employee_stats)
    
    print(f"\nWorkload Statistics:")
    print(f"  Average hours per active employee: {employee_stats['Total Hours'].mean():.1f}")
    print(f"  Max hours assigned: {employee_stats['Total Hours'].max():.1f}")
    print(f"  Min hours assigned: {employee_stats['Total Hours'].min():.1f}")
    print(f"  Std deviation: {employee_stats['Total Hours'].std():.1f}")
    
    # 5.3: Daily Staffing
    print("\n 5.3: Daily Staffing Levels")
    daily_stats = df_schedule.groupby('Day').agg({
        'Employee': 'count',
        'Hours': 'sum',
        'Cost': 'sum'
    })
    daily_stats.columns = ['Staff Assignments', 'Total Hours', 'Total Cost']
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    daily_stats = daily_stats.reindex(day_order)
    print(daily_stats)
    
    # 5.4: Shift Coverage Check
    coverage_issues = []
    understaffed_shifts = []
    
    for shift in shifts:
        shift_assignments = df_schedule[df_schedule['Shift'] == shift]
        required = shift_requirements[shift]
        total_required = sum(required.values())
        total_assigned = len(shift_assignments)
    
        if total_assigned < total_required:
            understaffed_shifts.append(
                f"{shift}: need {total_required}, have {total_assigned}"
            )
    
        for skill, req_count in required.items():
            if req_count > 0:
                assigned_with_skill = sum(
                    1 for _, row in shift_assignments.iterrows()
                    if skill in employees[row['Employee']]['skills']
                )
                if assigned_with_skill < req_count:
                    coverage_issues.append(
                        f"{shift} - {skill}: need {req_count}, have {assigned_with_skill}"
                    )
        
    # 5.5: Sample Schedules
    print("\n 5.5: Sample Weekly Schedules")
    for emp in list(employees.keys())[:3]:
        emp_schedule = df_schedule[df_schedule['Employee'] == emp].sort_values('Shift')
        if len(emp_schedule) > 0:
            print(f"\n{emp}:")
            print(f"  Skills: {', '.join(employees[emp]['skills'])}")
            print(f"  Total Hours: {emp_schedule['Hours'].sum()}")
            print(f"  Shifts:")
            for _, row in emp_schedule.iterrows():
                print(f"    - {row['Shift']} ({row['Hours']}h, Pref: {row['Preference']})")
    
    # 5.6: Shift Details (Sample)
    print("\n 5.6: Shift-by-Shift Breakdown (Monday)")
    monday_shifts = [s for s in shifts if s.startswith('Monday')]
    for shift in monday_shifts:
        shift_staff = df_schedule[df_schedule['Shift'] == shift]
        print(f"{shift} ({len(shift_staff)} staff):")
        for _, row in shift_staff.iterrows():
            print(f"  - {row['Employee']} ({row['Skills']})")
    
    # 5.7: Export Results
    print("\n 5.7: Exporting Results")
    df_schedule.to_csv('coffee_shop_schedule.csv', index=False)
    employee_stats.to_csv('employee_workload_summary.csv')
    daily_stats.to_csv('daily_staffing_summary.csv')
    
    # 5.8: Final Summary
    print("\n 5.8: Final Summary Statistics")
    print(f"\n OPTIMIZATION RESULTS:")
    print(f"  • Objective Value: {model.ObjVal:.2f}")
    print(f"  • Total Labor Cost: ${df_schedule['Cost'].sum():.2f}")
    print(f"  • Total Hours Scheduled: {df_schedule['Hours'].sum():.0f}")
    print(f"  • Avg Employee Satisfaction: {df_schedule['Preference'].mean():.2f}/5.0")
    print(f"  • Employees Utilized: {df_schedule['Employee'].nunique()}/{num_employees}")
    print(f"  • Cost per Hour: ${df_schedule['Cost'].sum() / df_schedule['Hours'].sum():.2f}")
    print(f"\n Solution is OPTIMAL")
    
elif model.status == GRB.INFEASIBLE:
    print("\n MODEL IS INFEASIBLE")
    model.computeIIS()
    model.write("coffee_shop_infeasible.ilp")
else:
    print(f"\n Optimization ended with status: {model.status}")

print("ANALYSIS COMPLETE")

COFFEE SHOP EMPLOYEE SCHEDULING OPTIMIZATION
MATHEMATICAL FORMULATION

SETS:
  E = Set of employees (e = 1, 2, ..., 15)
  S = Set of shifts (s = Monday 7-11AM, Monday 11AM-3PM, ..., Sunday 7-12AM)
  K = Set of skills (k = Barista, Cashier, Shift Lead, Food Prep, Cleaner)

PARAMETERS:
  w_e = Hourly wage of employee e ($/hour)
  h_s = Duration of shift s (hours)
  p_es = Preference score of employee e for shift s [1-5]
  a_es = Availability: 1 if employee e can work shift s, 0 otherwise
  skill_ek = 1 if employee e has skill k, 0 otherwise
  req_sk = Number of employees with skill k required for shift s
  maxH_e = Maximum weekly hours for employee e (hours)
  λ = Weight parameter for cost vs. preference trade-off [0,1]

DECISION VARIABLES:
  x_es ∈ {0,1} = 1 if employee e is assigned to shift s, 0 otherwise

OBJECTIVE FUNCTION:
  Minimize: λ × (Total Cost) - (1 - λ) × (Total Preference)
  
  Where:
    Total Cost = Σ_e Σ_s (x_es × w_e × h_s)
    Total Preference = Σ_e Σ_s (x_es × p_es)
